In [1]:

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.framework import ops
import tensorflow as tf
from itertools import cycle
from keras.layers import BatchNormalization

In [8]:
valid_datagen = ImageDataGenerator(rescale=1./255)
valid_set = valid_datagen.flow_from_directory('validation',
                                                 target_size=(256, 256),
                                                 batch_size=32,
                                                 class_mode='binary')

Found 400 images belonging to 2 classes.


In [9]:
train_datagen = ImageDataGenerator(rescale=1./255)
training_set = train_datagen.flow_from_directory('training',
                                                 target_size=(256, 256),
                                                 batch_size=32,
                                                 class_mode='binary')

Found 2450 images belonging to 2 classes.


In [21]:

import sys
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.optimizers import SGD



model = Sequential()
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=(256, 256, 3)))
model.add(Conv2D(32, (3, 3), activation='relu',  padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu',  padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3), activation='relu',  padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation=tf.nn.sigmoid))

In [22]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 256, 256, 16)      448       
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 256, 256, 32)      4640      
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 256, 256, 32)      9248      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 128, 128, 32)      0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 128, 128, 32)      0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 128, 128, 32)      128       
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 128, 128, 64)     

In [23]:
model.compile(optimizer = tf.optimizers.Adam(),
              loss = 'binary_crossentropy',
              metrics=['accuracy'])

In [25]:


history = model.fit(training_set,
      steps_per_epoch=15,  
      epochs=15,
      verbose=1,
      validation_data = valid_set,
      validation_steps=15)

Epoch 1/15
15/15 [==============================] - 53s 4s/step - loss: 0.4745 - accuracy: 0.7811
Epoch 2/15
15/15 [==============================] - 56s 4s/step - loss: 0.4438 - accuracy: 0.8146
Epoch 3/15
15/15 [==============================] - 56s 4s/step - loss: 0.4165 - accuracy: 0.8396
Epoch 4/15
15/15 [==============================] - 54s 4s/step - loss: 0.3786 - accuracy: 0.8455
Epoch 5/15
15/15 [==============================] - 56s 4s/step - loss: 0.3554 - accuracy: 0.8498
Epoch 6/15
15/15 [==============================] - 54s 4s/step - loss: 0.3367 - accuracy: 0.8519
Epoch 7/15
15/15 [==============================] - 56s 4s/step - loss: 0.3433 - accuracy: 0.8458
Epoch 8/15
15/15 [==============================] - 56s 4s/step - loss: 0.3724 - accuracy: 0.8229
Epoch 9/15
15/15 [==============================] - 56s 4s/step - loss: 0.3258 - accuracy: 0.8687
Epoch 10/15
15/15 [==============================] - 56s 4s/step - loss: 0.2976 - accuracy: 0.8646
Epoch 11/15
15/15 [

In [26]:
import os
os.chdir('test')

In [27]:
import random
import json
# from utils.io import write_json

def write_json(filename, result):
    with open(filename, 'w') as outfile:
        json.dump(result, outfile)

def read_json(filename):
    with open(filename, 'r') as outfile:
        data =  json.load(outfile)
    return data

def generate_sample_file(filename):
    res = {}
    for i in range(1,99):
        test_set = str(i) + '.jpg'
        test_image = image.load_img(test_set, target_size=(256, 256))
        test_image = image.img_to_array(test_image)
        test_image = np.expand_dims(test_image, axis=0)
        result= model.predict(test_image)
        if int(result[0][0])==1:
            res[test_set]=0
        else:
            res[test_set]=1

    write_json(filename, res)

if __name__ == '__main__':
    generate_sample_file('./sample_result1.json')
